In [1]:
#!pip install xmltodict

In [1]:
import xmltodict
import json
import pandas as pd

In [2]:
periods = [1980, 1985, 1990, 1995, 2000, 2005, 2010, 2015]

all_data = {}

In [3]:
for period in periods:
    period = str(period)

    with open('data/TheNewYorkTimes.' + period +'.xml') as xml_file:
        all_data[period] = xmltodict.parse(xml_file.read())

In [4]:
len(all_data['1980']['records']['record'])

94697

In [5]:
for period in periods:
    period = str(period)

    a = len(all_data[period]['records']['record'])
    data = {}

    for i in range(a):
        data[i] = all_data[period]['records']['record'][i]

    all_data[period] = pd.DataFrame.from_dict(data, orient='index')


In [6]:
for period in periods:
    period = str(period)
    all_data[period] = all_data[period].fillna('')

In [7]:
all_data[period].head(1)

,record_id,source_id,publishDate,series,series2,title,author,leadtext,fulltext,link
0,5F0R-HCS1-DXY4-X2RB,https://www.nexis.com/docview/getDocForCuiReq?...,20150105T000000Z,The New York Times,Section B; Column 0; Business/Financial Desk; ...,"Tax Increase on Digital Goods to Hit Europe, T...",By MARK SCOTT,LONDON -- Europe's tax showdown could be heade...,What remains unclear is who in the 28-country ...,http://www.nytimes.com/2015/01/02/business/int...


In [8]:
for period in periods:
    period = str(period)
    all_data[period]['year'] = list(map(lambda x: int(x[:4]), all_data[period]['publishDate']))

    all_data[period]['text'] = all_data[period]['leadtext'] + all_data[period]['fulltext']

    all_data[period].drop(['source_id', 'publishDate', 'series', 'series2', 'author', 'title', 'link', 'leadtext', 'fulltext'], axis = 1, inplace= True)

In [9]:
for period in periods:
    period = str(period)
    all_data[period] = all_data[period][all_data[period]['year'] == int(period)]

In [10]:
for period in periods:
    period = str(period)

    all_data[period]['word_count'] = list(map(lambda x: len(str(x).split()), all_data[period]['text']))

    all_data[period] = all_data[period][all_data[period]['word_count'] > 0]

In [11]:
all_data[period].iloc[0]['text']

"LONDON -- Europe's tax showdown could be headed straight to people's wallets. With the new year, a change in fiscal rules in the European Union is increasing the tax on many purchases of digital content like e-books and smartphone applications. Under the new rules, first approved in 2008, the tax rate on digital services like cloud storage and movie streaming will be determined by where consumers live, and not where the company selling the product has its European headquarters. Tax experts say Europe's revamped rules could add up to an extra $1 billion in annual tax revenue for European governments.What remains unclear is who in the 28-country bloc will pay most of the bill. ''There inevitably will be a price change,'' said Richard Mollet, chief executive of the Publishers Association, a British trade body. ''The question is whether retailers, publishers or customers will have to take on board any increase.'' The changes to Europe's so-called value-added tax -- a tax on goods and serv

# Pre-processing 

In [12]:
from nltk.stem import WordNetLemmatizer
import re
import numpy as np
import nltk

lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
stop_words = list(set(stopwords.words('english')))

In [13]:
def preprocess(text_string):
    
    non_word_char = "[^\w\']"
    
    parsed_text = re.sub(non_word_char, ' ', text_string)

    parsed_text = parsed_text.lower()

    parsed_text = parsed_text.split()

    parsed_text = list(filter(lambda x: not x in stop_words, parsed_text))

    parsed_text = list(map(lambda x: lemmatizer.lemmatize(x), parsed_text))

    parsed_text = ' '.join(parsed_text)
    
    return parsed_text

In [14]:
all_data['1980']['text'] = np.vectorize(preprocess)(all_data['1980']['text'])
all_data['1980'].to_csv('data_csv/news_1980.csv')

In [15]:
all_data['1985']['text'] = np.vectorize(preprocess)(all_data['1985']['text'])
all_data['1985'].to_csv('data_csv/news_1985.csv')

In [16]:
all_data['1990']['text'] = np.vectorize(preprocess)(all_data['1990']['text'])
all_data['1990'].to_csv('data_csv/news_1990.csv')

In [17]:
all_data['1995']['text'] = np.vectorize(preprocess)(all_data['1995']['text'])
all_data['1995'].to_csv('data_csv/news_1995.csv')

In [18]:
all_data['2000']['text'] = np.vectorize(preprocess)(all_data['2000']['text'])
all_data['2000'].to_csv('data_csv/news_2000.csv')

: 

: 

In [ ]:
all_data['2005']['text'] = np.vectorize(preprocess)(all_data['2005']['text'])
all_data['2005'].to_csv('data_csv/news_2005.csv')

In [ ]:
all_data['2010']['text'] = np.vectorize(preprocess)(all_data['2010']['text'])
all_data['2010'].to_csv('data_csv/news_2010.csv')

In [ ]:
all_data['2015']['text'] = np.vectorize(preprocess)(all_data['2015']['text'])
all_data['2015'].to_csv('data_csv/news_2015.csv')

# Getting the list of target words:

In [20]:
from collections import Counter

In [ ]:
def word_freq(words):
    word_frequencies = Counter(words)

    return word_frequencies

In [ ]:
words = {str(k): [] for k in periods}

for period in periods:
    period = str(period)

    for text in all_data[period]['text']:
        words[period] += str(text).split()

In [23]:
def most_common_tokens(n, words):

    counter = Counter(words)

    return counter.most_common(n)

In [33]:

words = {str(k): [] for k in periods}

for period in periods:
    period = str(period)

    for text in all_data[period]['text']:
        words[period] += str(text).split()


In [38]:
most_common_words = {str(k): [] for k in periods}

for period in periods:
    period = str(period)

    most_common_words[period] = most_common_tokens(100, words[period])

In [39]:
most_common_words['2005']

[('Mr.', 266915),
 ('The', 260035),
 ('said', 217372),
 ('--', 145759),
 ('I', 115304),
 ('would', 112121),
 ('said.', 109291),
 ('one', 94649),
 ('But', 88252),
 ('like', 78475),
 ('In', 75035),
 ('New', 75030),
 ('He', 73705),
 ('also', 68677),
 ('two', 67751),
 ('last', 65084),
 ('Ms.', 58517),
 ('first', 56976),
 ('new', 55830),
 ('could', 55631),
 ('people', 55449),
 ('said,', 51659),
 ("''I", 48059),
 ('many', 44945),
 ('It', 43293),
 ('A', 42424),
 ('years', 41151),
 ('time', 38552),
 ('even', 38318),
 ('made', 38156),
 ("''The", 38059),
 ('American', 36842),
 ('three', 36513),
 ('percent', 36216),
 ('get', 35560),
 ('make', 35222),
 ('much', 35100),
 ('And', 35095),
 ('York', 34424),
 ('United', 34313),
 ('million', 33168),
 ('may', 32105),
 ('still', 29155),
 ('year', 28269),
 ('going', 28030),
 ('way', 27651),
 ('back', 27171),
 ('Dr.', 27112),
 ('since', 26444),
 ('company', 25627),
 ('say', 25611),
 ('They', 25183),
 ('work', 25134),
 ('officials', 24386),
 ('think', 24300)